# STK Object Model Walkthrough using Python

## Uncomment and run the cell below the first time you connect to STK with Python on a new computer

In [ ]:
# # Import the comtypes package to connect to an external application
# import comtypes
# from comtypes.client import CreateObject

# # Create a new instance of STK
# app = CreateObject('STK11.Application')
# app.Visible = True # Needed to view the user interface application

# # Get the IAgStkObjectRoot interface
# root = app.Personality2

# # Note: When 'root=uiApplication.Personality2' is executed, the comtypes library automatically creates a gen folder that contains
# # STKObjects and other Python wrappers for the STK libraries. After running this at least once on your computer, the libraries 
# # should be imported before the 'uiApplication=CreateObject("STK11.Application")' line for improved performance.

# # The Astrogator wrapper needs to specifically be loaded with the line below, then AgStkGatorLib is created.
# comtypes.client.GetModule((comtypes.GUID("{090D317C-31A7-4AF7-89CD-25FE18F4017C}") ,1,0))

# # View the location and newly generated wrappers
# import os as os
# print(comtypes.client.gen_dir, '\n')
# print(os.listdir(comtypes.client.gen_dir))

# # After running this cell comment out this cell.
# # Use "Ctrl" + "a" to select all of the cell content, then Use "Ctrl" + "/" to toggle comments in Jupyter Notebooks

## After running the cell above, you can import the packages directly

## A line of code must be added where the comments say ACTION ##

In [ ]:
# Import the needed packages
import comtypes
from comtypes.client import CreateObject
from comtypes.client import GetActiveObject
from comtypes.gen import STKObjects
from comtypes.gen import AgSTKVgtLib

In [ ]:
# Create a new instance of STK
app = CreateObject('STK11.Application')
app.Visible = True
app.UserControl= True
# Alternatively attach to an existing instance of STK, use the handle app
# app = GetActiveObject('STK11.Application')

## Build the scenario ## 

In [ ]:
# Get the IAgStkObjectRoot interface, use the handle root
root = app.Personality2
# Personality2 is AGI's way for refering to the 2nd implementation of the Object Model

In [ ]:
# Create a new scenario called 'STKPythonTutorial' with the handle sc
root.NewScenario('STKPythonTutorial')
# Attach to an existing Scenario
sc = root.CurrentScenario

In [ ]:
# Reposition the STK application
from win32api import GetSystemMetrics
app.Top=0
app.Left=0
app.Width=int(GetSystemMetrics(0)/2)
app.Height=int(GetSystemMetrics(1)-30)

In [ ]:
# View the classes of each object
print(type(app))
print(type(root))
print(type(sc))
# The IAgSTKObject contains properties generic to all STK objects

In [ ]:
# Cast sc into an IAgScenario called sc2 to get Scenario specific properties
sc2 = sc.QueryInterface(STKObjects.IAgScenario)
# Double check the type is now an IAgScenario
type(sc2)

In [ ]:
# Set the Scenario time period
time1 = 'Today'
time2 = '+24'
# ACTION: Use time1 and time2 to set the scenario time period 


In [ ]:
# Rewind the scenario to the start time
root.Rewind();

### Add a Facility ###

In [ ]:
# Add a new facility called 'AGI_Greenbelt' with the handle fac
fac = sc.Children.New(STKObjects.eFacility,'AGI_Greenbelt')
# Cast fac into an IAgFacility called fac2
fac2 = fac.QueryInterface(STKObjects.IAgFacility)

In [ ]:
# Assign a new geodetic position
lat = 39.0095
lon = -76.896
alt = 0
# ACTION: Set the facility's location using lat, lon and alt


### Add Satellites ###

In [ ]:
# Add a new satellite called 'LEOSat' with the handle sat
sat = sc.Children.New(STKObjects.eSatellite, 'LEOSat')
# Get the IAgSatellite interface with the handle sat2
sat2 = sat.QueryInterface(STKObjects.IAgSatellite)

In [ ]:
# View Supported Propagator Types
sat2.PropagatorSupportedTypes

In [ ]:
# Set the Propagator to J2Perturbation
sat2.SetPropagatorType(STKObjects.ePropagatorJ2Perturbation);
# Grab a handle for the satellite propagator called prop
prop = sat2.Propagator
# View the type of the propagator
type(prop)
# Check the Programming Help for the class type

In [ ]:
# Cast prop to the IAgVePropagatorJ2Perturbation interface with the handle prop2
prop2 = prop.QueryInterface(STKObjects.IAgVePropagatorJ2Perturbation)

In [ ]:
# Propagate the satellite. The UI will now reflect the changes to the propagator
prop2.Propagate();

In [ ]:
# Update the satellite's Epoch 
epoch = '17 Sep 2018 00:00:00.000'
prop2.InitialState.Epoch = epoch

In [ ]:
# Determine the type of prop2.InitialState.Representation
type(prop2.InitialState.Representation)
# Look up the class in the Programming Help

In [ ]:
# Assign the satellite's orbit state using the classical orbital elements in the ICRF Coordinate System
coordSys = 11
sma = 7000
e = 0.01
i = 90
aop = 270
raan = 90
ma = 10
# ACTION: Assign the orbital elements and propagate the satellite.

# ACTION: Propagate the satellite


In [ ]:
# Use the New Connect Command to create a 2nd satellite called ConnectSat. Reference the Programming Help
cmd = 'New / */Satellite ConnectSat'
root.ExecuteCommand(cmd);

In [ ]:
# Use the following command to set the state of ConnectSat
# ACTION: Use the scenario start and stop time in the connect command. Fill in the blanks.
cmd = 'SetState */Satellite/ConnectSat Classical TwoBody "' +  + '" "'+  +'" 60 ICRF "'+  + '" 7000000.0 0.01 90 270 0 10.0'
print(cmd)
# ACTION: Execute the Connect Command


In [ ]:
# Use the root to get a handle called conSat for ConnectSat by getting the object from a file path
conSat = root.GetObjectFromPath('*/Satellite/ConnectSat')

In [ ]:
# ACTION: Grab the IAgSatellite interface called conSat2


In [ ]:
# Change ConnectSat's color
basicAtt = conSat2.Graphics.Attributes.QueryInterface(STKObjects.IAgVeGfxAttributesBasic)
basicAtt.Color = 16776960 # hexadecimal to decimal of the BGR value

## Using Analysis Workbench ##

In [ ]:
# Create a new vector from the vector factory
vecFac = sat.Vgt.Vectors.Factory
type(vecFac)
# Create a new displacement vector
try:
    toFac = vecFac.Create('ToAGI_Greenbelt','Description',AgSTKVgtLib.eCrdnVectorTypeDisplacement)
except:
    toFac = sat.Vgt.Vectors.Item('ToAGI_Greenbelt')
type(toFac)

In [ ]:
# ACTION: Get a handle for the interface allowing you to access the displacement vector properties 


In [ ]:
# Ensure the vector goes from LEOSat to AGI_Greenbelt
toFac2.Origin.SetPath('Satellite/LEOSat Center')
toFac2.Destination.SetPath('Facility/AGI_Greenbelt Center');

In [ ]:
# Add the newly created vector to LEOSat
vecVO = sat2.VO.Vector.RefCrdns.Add(0,'Satellite/LEOSat ToAGI_Greenbelt')
vecVO.Visible = 1

## Calculate Access  ##

In [ ]:
# Get access to AGI_Greenbelt
access = sat.GetAccessToObject(fac2)

In [ ]:
# ACTION: Compute the access


In [ ]:
# Push the computed access intervals into an array called accessTimes
accessTimes = access.ComputedAccessIntervalTimes.ToArray(0, -1)
# If the types are not all numeric, then the array will actually be a tuple

In [ ]:
# Import numpy and pandas to help out
import numpy as np
import pandas as pd

In [ ]:
# Converts the tuple into an array and then displays the data
accessTimes = np.asarray(accessTimes)
df = pd.DataFrame({'Start Time':accessTimes[:,0],'Stop Time':accessTimes[:,1]})
df

## Pulling Data Providers ##

In [ ]:
# Extract the folder called 'Cartesian Velocity' into the handle cartVel
cartVel = sat.DataProviders.Item("Cartesian Velocity")
# View the type of cartVel
type(cartVel)

In [ ]:
# Cast cartVel into an IAgDataProviderGroup
cartVel = cartVel.QueryInterface(STKObjects.IAgDataProviderGroup)

In [ ]:
# Grab a handle for the ICRF folder called CartVelICRF
cartVelICRF = cartVel.Group.Item("ICRF")

In [ ]:
# ACTION: Recast cartVelICRF to an IAgDataPrvTimeVar

type(cartVelICRF)

In [ ]:
# Extract the Time, speed, radial and in-track data providers. Save the output as result
elems = ['Time','speed','radial','in-track']
startTime = sc2.StartTime
stopTime = sc2.StopTime
timeStep = 60

# ACTION: Execute the desired elements, using the scenario start and stop time and with the desired time step


In [ ]:
# Grab each the values from each DataSet
time = result.DataSets.Item(0).GetValues()
speed = result.DataSets.Item(1).GetValues()
radial = result.DataSets.Item(2).GetValues()
intrack = result.DataSets.Item(3).GetValues()

## Viewing Results ##

In [ ]:
# Use Pandas to show the Satellite Velocity Data
df = pd.DataFrame({'Time':time,'Speed':speed,'Radial':radial,'In-Track':intrack});
df.head(10)

In [ ]:
# Import matlplotlib.pyplot to plot the speed
import matplotlib.pyplot as plt

In [ ]:
# Plot the speed vs time
plt.plot(speed)
plt.xlabel('Time [mins]')
plt.ylabel('[km/sec]')
plt.title('Speed vs Time')
plt.show()

## Save and Close ##

In [ ]:
# Create a folder to save the scenario and contents
# import os as os
# userDir = root.ExecuteCommand('GetDirectory / DefaultUser').Item(0)
# saveFolder =  userDir + '\\' + sc.InstanceName
# os.mkdir(saveFolder)
# # Save the scenario
# root.SaveAs(saveFolder + '\\' + sc.InstanceName)
# print(saveFolder)

In [ ]:
# Delete the handles and close STK
# del root; 
# app.Quit();
# del app;